# **DataLoader**

Ya que vimos el Procesamiento necesario para limpiar al maximo nuestro dataset, ahora veamos como podemos contruir un DataLoader en Pytorch.

## **¿Qué es un DataLoader?**

Un DataLoader lo usamos para cargar nuestros datos en batches, transformarlos y los mezcla para un entrenamiento eficiente.

In [1]:
# Las librerias que usaremos

import torchtext
print(torchtext.__version__)

c:\Users\KATANA\Desktop\git\Personal\nlp-proyecto13\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.17.0+cpu


In [14]:
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
from datasets import load_dataset
import random

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace 

## Conjunto de Datos

 Es una colección de muestra de datos, se usa para dar entrada al dataloader con ciertas transformaciones previas

In [ ]:
# Cargamos los datos

dataset = load_dataset("cnn_dailymail", "3.0.0")

# Creamos los iteradores por cada set de datos que cargamos

train_iter = iter(dataset['train'])
validation_iter = iter(dataset['validation'])
test_data_iter = iter(dataset['test'])

In [ ]:
for n in range(5):
    
    value = next(train_iter) #article, highlights
    
    print(f"muestra {str(n+1)}")
    print(f'Articulo: {value["article"]} \nResumen: {value["highlights"]}')
    print()

muestra 1
Articulo: BAGHDAD, Iraq (CNN) -- The women are too afraid and ashamed to show their faces or have their real names used. They have been driven to sell their bodies to put food on the table for their children -- for as little as $8 a day. Suha, 37, is a mother of three. She says her husband thinks she is cleaning houses when she leaves home. "People shouldn't criticize women, or talk badly about them," says 37-year-old Suha as she adjusts the light colored scarf she wears these days to avoid extremists who insist women cover themselves. "They all say we have lost our way, but they never ask why we had to take this path." A mother of three, she wears light makeup, a gold pendant of Iraq around her neck, and an unexpected air of elegance about her. "I don't have money to take my kid to the doctor. I have to do anything that I can to preserve my child, because I am a mother," she says, explaining why she prostitutes herself. Anger and frustration rise in her voice as she speaks. 

## Configuración del tokenizador

El tokenizador que usaremos sera el de huggin face que por detras usa el algoritmo BPE que usa el enfoque botton-up, es decir empieza desde unidades individuales del lenguaje como los caracteres en ingles, luego junta los pares mas frecuentes de manera iterativa hasta completar los tokens.